### Ingestión del archivo "movie_genre.json"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Paso 1 - Leer archvio JSON usando "DataFrameReader" de Spark

In [0]:
movie_genre_schema = "movieId INT, genreId STRING"

In [0]:
movie_genre_df = spark.read \
    .schema(movie_genre_schema) \
    .json(f"{bronze_folder_path}/{v_file_date}/movie_genre.json")


#### Paso 2 - Cambiar el nombre de las columnas y agregar la columnas "ingestion_date" y "environment" al data frame

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
movie_genre_final_df = add_ingestion_date(movie_genre_df) \
    .withColumnRenamed("movieId","movie_id") \
    .withColumnRenamed("genreId","genre_id") \
    .withColumn("environment", lit(v_environment)) \
    .withColumn("file_date", lit(v_file_date))

#### Paso 4 - Escribir datos en el DataLake en formato "Parquet"

In [0]:
#overwrite_partition(movie_genre_final_df, "movie_silver","movies_genres", "file_date")

In [0]:
merge_condition = "tgt.movie_id = src.movie_id AND tgt.genre_id = src.genre_id AND tgt.file_date = src.file_date"
merge_delta_lake(movie_genre_final_df, "movie_silver","movies_genres", silver_folder_path, merge_condition, "file_date")
#movie_genre_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.movies_genres")
#movie_genre_final_df.write.mode("overwrite").partitionBy("movie_id").parquet(f"{silver_folder_path}/movies_genres")
#movie_genre_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/movies_genres")

In [0]:
%sql
SELECT file_date, COUNT(1) 
FROM movie_silver.movies_genres
GROUP BY file_date;

In [0]:
%sql
SELECT * FROM movie_silver.movies_genres

In [0]:
#display(spark.read.parquet(f"{silver_folder_path}/movies_genres"))

In [0]:
dbutils.notebook.exit("Success")